## Import Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
# from numpy.linalg import inv as inv

#Import function to get the covariate matrix that includes spike history from previous bins
from preprocessing_funcs import get_spikes_with_history

#Import decoder functions
import Decoder_funcs
from Decoder_funcs import get_vaf

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
Using gpu device 3: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/opt/anaconda/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## Import Data

**Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin**

**The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"**

In [3]:
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/'
folder='/home/jglaser2/Data/DecData/'

with open(folder+'test_data.pickle','rb') as f:
    neural_data,vels_binned=pickle.load(f)

## Define Covariates ##

### User Options: Define what time period to use spikes from

In [4]:
lag=-2 #What time bin of spikes should be used relative to the output
#(lag=-1 means use the spikes 1 bin before the output)

### Get Input Covariates

In [5]:
#Format for Kalman filter
X_kf=neural_data

### Get Output Covariates

In [6]:
#Set decoding output for the KF
pos=np.zeros(vels_binned.shape)
pos[0,:]=0
for i in range(pos.shape[0]-1):
    pos[i+1,0]=pos[i,0]+vels_binned[i,0]*.05
    pos[i+1,1]=pos[i,1]+vels_binned[i,1]*.05
    
temp=np.diff(vels_binned,axis=0)
acc=np.concatenate((temp,temp[-1:,:]),axis=0)

y_kf=np.concatenate((pos,vels_binned,acc),axis=1)

### Process Covariates

In [7]:
def zscore_nan(X,axis):
    X_zscore=(X - np.nanmean(X,axis=axis)) / np.nanstd(X,axis=axis)
    return X_zscore

In [8]:
#Normalize inputs
X_kf=zscore_nan(X_kf,axis=0)

#Zero-center outputs
y_kf_mean=np.mean(y_kf,axis=0)
y_kf=y_kf-y_kf_mean

In [10]:
#More Kalman filter preprocessing
num_examples=X_kf.shape[0]

#Re-align data to take lag into account
if lag<0:
    y_kf=y_kf[-lag:,:]
    X_kf=X_kf[0:num_examples+lag,:]
if lag>0:
    y_kf=y_kf[0:num_examples-lag,:]
    X_kf=X_kf[lag:num_examples,:]

## Split into training/testing/validation sets

### Remove Validation Data

In [12]:
#What part of the data you're using (remove the validation set that we fit parameters on)
use_range=[0, 0.85]
num_examples_orig=X_kf.shape[0]
#Below starts at "bins_before" and ends with a boundary of "bins_after"
use_set=np.arange(np.int(np.round(use_range[0]*num_examples_orig)),np.int(np.round(use_range[1]*num_examples_orig)))
X_kf=X_kf[use_set,:]
y_kf=y_kf[use_set,:]

### Split Data: For KF

In [11]:

        
#Note that each range has a buffer of 1 bin at the beginning and end
#This makes it so that the different sets don't include overlapping data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples_kf))+1,np.int(np.round(training_range[1]*num_examples_kf))-1)
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples_kf))+1,np.int(np.round(testing_range[1]*num_examples_kf))-1)
valid_set=np.arange(np.int(np.round(valid_range[0]*num_examples_kf))+1,np.int(np.round(valid_range[1]*num_examples_kf))-1)

#Get training data
X_kf_train=X_kf[training_set,:]
y_kf_train=y_kf[training_set,:]

#Get testing data
X_kf_test=X_kf[testing_set,:]
y_kf_test=y_kf[testing_set,:]

#Get validation data
X_kf_valid=X_kf[valid_set,:]
y_kf_valid=y_kf[valid_set,:]

In [19]:
num_cv=10
cv_frac=num_cv**-1

#Number of examples after taking into account bins removed for lag alignment
num_examples=X_kf.shape[0]

mean_vafs_kf=np.empty(num_cv)

for i in range(2):#(num_cv):
    testing_range=[i*cv_frac,(i+1)*cv_frac]
    testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples)),np.int(np.round(testing_range[1]*num_examples)))
    training_set=np.arange(num_examples) #Initialize training set as all examples
    training_set=np.delete(training_set,testing_set) #Remove testing set from all examples to create training set
    #Now make sure there's no overlap between the training/testing sets by removing border entries from test set:
    rmv_before=np.arange(0,1)
    testing_set=np.delete(testing_set,rmv_before)
    rmv_after=np.arange(testing_set.shape[0]-1,testing_set.shape[0])
    testing_set=np.delete(testing_set,rmv_after)
    
    ###Split Data###
    X_kf_train=X_kf[training_set,:]
    y_kf_train=y_kf[training_set,:]

    X_kf_test=X_kf[testing_set,:]
    y_kf_test=y_kf[testing_set,:]
    
    
    ##### DECODERS ########
    
    ###WIENER FILTER###
    model_kf=Decoder_funcs.kf_model(X_kf_train,y_kf_train)
    y_test_pred_kf=Decoder_funcs.kf_predict(model_kf,X_kf_test,y_kf_test)
    vafs_kf=get_vaf(y_kf_test,y_test_pred_kf)
    mean_vafs_kf[i]=np.mean(vafs_kf[2:4])

In [20]:
mean_vafs_kf

array([ 0.77626748,  0.80713614,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])